This notebook focuses on analyzing candlestick patterns using the TA-Lib library and visualizing them on a candlestick chart. Below is a summary of each code block's purpose:

## 1. Imports and Setup
- **Purpose**: Import the necessary libraries, such as `pandas` for data manipulation, `plotly.graph_objs` for visualization, and `talib` for technical analysis.
- **Libraries**:
  - `pandas`: For handling and processing the stock data.
  - `plotly.graph_objs`: For creating interactive visualizations, specifically candlestick charts.
  - `talib`: For calculating various candlestick patterns.


In [237]:
import pandas as pd
import yfinance as yf
import talib

## 2. Loading Data
- **Purpose**: Load historical stock data (e.g., Nifty index data) into a Pandas DataFrame.
- **Data Structure**:
  - Columns typically include `Date`, `Open`, `High`, `Low`, and `Close`.

In [238]:
# Define the ticker symbol for NIFTY 50
nifty_ticker = "^GSPC"

# Download historical data for NIFTY 50
df = yf.download(nifty_ticker, start="2024-07-10", end="2024-08-14", interval='5m')

# Display the first few rows of the data
df.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-07-10 09:30:00-04:00,5591.259766,5592.669922,5588.720215,5589.629883,5589.629883,0
2024-07-10 09:35:00-04:00,5589.799805,5589.810059,5587.080078,5589.149902,5589.149902,36275310
2024-07-10 09:40:00-04:00,5589.020020,5590.529785,5588.479980,5589.669922,5589.669922,35772274
2024-07-10 09:45:00-04:00,5589.529785,5593.350098,5589.459961,5593.040039,5593.040039,35938266
2024-07-10 09:50:00-04:00,5593.049805,5593.120117,5590.560059,5592.740234,5592.740234,31559790


In [239]:
op = df['Open'].astype(float)
hi = df['High'].astype(float)
lo = df['Low'].astype(float)
cl = df['Close'].astype(float)

## 3. Calculating Candlestick Patterns
- **Purpose**: Use TA-Lib to identify specific candlestick patterns in the stock data.
- **Key Patterns**:
  - `DOJI`
  - `Morning Star`
  - `Hammer`
  - `Morning Doji Star`
  - `Engulfing`
- **Output**: New columns are added to the DataFrame, with values of 100 indicating the presence of these patterns.

In [255]:
df['DOJI'] = talib.CDLDOJI(op, hi, lo, cl)
df['Engulfing'] = talib.CDLENGULFING(op, hi, lo, cl)
df['HAMMER'] = talib.CDLHAMMER(op, hi, lo, cl)
df['HANGINGMAN'] = talib.CDLHANGINGMAN(op, hi, lo, cl)
df['HARAMI'] = talib.CDLHARAMI(op, hi, lo, cl)
df['MORNINGSTAR'] = talib.CDLMORNINGSTAR(op, hi, lo, cl)





In [267]:

# Counting occurrences of each pattern
pattern_counts = df[['DOJI', 'Engulfing', 'HAMMER', 'HANGINGMAN', 
                     'HARAMI', 
                     'MORNINGSTAR']].apply(lambda x: x[x != 0].count())

print("Candlestick Patterns Identified Across the Given Period:")
print(pattern_counts)


Candlestick Patterns Identified Across the Given Period:
DOJI           244
Engulfing      224
HAMMER          41
HANGINGMAN      39
HARAMI         116
MORNINGSTAR      9
dtype: int64


## 4. Plotting the Candlestick Chart
- **Purpose**: Create a candlestick chart using Plotly to visualize the stock data.
- **Visualization**:
  - The chart displays Open, High, Low, and Close values.
  - Annotations are added above candles where specific patterns are detected.

In [264]:
import plotly.graph_objects as go

# Assuming df_test is your DataFrame with a DateTime index
fig = go.Figure(data=[go.Candlestick(x=df_test.index,
                                     open=df_test['Open'],
                                     high=df_test['High'],
                                     low=df_test['Low'],
                                     close=df_test['Close'],
                                     name='Nifty Candlestick')])

# Add annotations for patterns
pattern_columns = ['Engulfing', 'HAMMER', 'MORNINGSTAR',
                    'HARAMI']

for idx in range(len(df_test)):
    for pattern in pattern_columns:
        if df_test.iloc[idx][pattern] != 0:
            fig.add_annotation(
                x=df_test.index[idx],
                y=df_test.iloc[idx]['High'] + (df_test.iloc[idx]['High'] - df_test.iloc[idx]['Low']) * 0.3,  # Adjust the offset to be 10% of the candle height
                text=pattern,
                showarrow=True,
                arrowhead=1,
                textangle=90,
                yshift=5  # Optional: Shift the text slightly down if needed
            )


fig.update_xaxes(
    rangeslider_visible=False,
    rangebreaks=[
        # Remove weekends
        dict(bounds=["sat", "mon"]),  # Skips Saturday and Sunday
        
        # Remove non-trading hours
        dict(bounds=[16, 9.5], pattern="hour"),  # Skips from 4:00 PM (16) to 9:30 AM (9.5)
        
        # Optionally, remove specific holidays by specifying exact dates
        # dict(values=["2024-01-01", "2024-12-25"])
    ]
)


# Customize the layout
fig.update_layout(title='S&P500 Candlestick Chart with Skipped Non-Trading Hours',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  width=1000,
                  height=600)

fig.update_layout(
    dragmode='zoom',  # Default interaction is zoom
    hovermode='x',    # Hover effect on the x-axis
    xaxis_rangeslider_visible=False  # Keep the range slider hidden
)

# Display the interactive plot
fig.show()



In [249]:
import vectorbt as vbt
import pandas as pd

from datetime import datetime, timedelta

buys = (df['HAMMER']==100)
sells = (df['HANGINGMAN']==-100)



In [251]:
pf = vbt.Portfolio.from_signals(df.Close, buys, sells, fees = 0.005)

In [252]:
print(pf.stats())

Start                         2024-07-10 09:30:00-04:00
End                           2024-08-13 15:55:00-04:00
Period                                             1950
Start Value                                       100.0
End Value                                     79.795154
Total Return [%]                             -20.204846
Benchmark Return [%]                          -2.781396
Max Gross Exposure [%]                            100.0
Total Fees Paid                               15.150903
Max Drawdown [%]                              20.204846
Max Drawdown Duration                            1885.0
Total Trades                                         17
Total Closed Trades                                  17
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                   5.882353
Best Trade [%]                                 0.476165
Worst Trade [%]                               -5

C:\Users\adiku\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

C:\Users\adiku\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

C:\Users\adiku\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

C:\Users\adiku\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



In [ ]:
import pandas as pd
import mplfinance as mpf

# Assuming df_test is already defined as a subset of your main data
df_test = df.tail(200)

# Define the figure and axes for plotting with increased candlestick width
fig, axlist = mpf.plot(df_test, type='candle', volume=True,
                       style='charles',
                       ylim=(df_test['Low'].min() - 50, df_test['High'].max() + 50),
                       title='Nifty Candlestick Chart with Annotations',
                       returnfig=True)  # Adjust the candle_width parameter to make the candlesticks larger

# Custom annotation logic based on candlestick patterns
pattern_columns = ['DOJI', 'Engulfing', 'HAMMER', 'MORNINGSTAR', 'HANGINGMAN',
                   '2CROWS', '3BLACKCROWS', '3WHITESOLDIERS', 'EVENINGSTAR', 'HARAMI']

for idx in range(len(df_test)):
    for pattern in pattern_columns:
        if df_test.iloc[idx][pattern] != 0:  # Check if the pattern is detected
            x = idx
            y = df_test.iloc[idx]['High'] + 50  # Adjust position above the high price
            axlist[0].text(x, y, pattern, fontsize=7, color='red', rotation=90, ha='center')

# Show the plot with annotations
mpf.show()